In [ ]:
# Allows imports from modules in the project directory
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.path.abspath('')).resolve()
MODULE_DIRECTORY = CURRENT_DIRECTORY.parent
PROJECT_DIRECTORY = MODULE_DIRECTORY.parents[1]
sys.path.extend([str(MODULE_DIRECTORY)])

print(f'Python {sys.version} on {sys.platform}')

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import tensorflow as tf

import utilities

In [ ]:
# Hyperparameter
LEARNING_RATE = 0.001
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100
EPOCHS = 5

wavelet_name = 'gaus5'
scales = 20

In [ ]:
# Load wavelet data
train_path = CURRENT_DIRECTORY / f'{wavelet_name}_{scales}/train.npz'
test_path = CURRENT_DIRECTORY / f'{wavelet_name}_{scales}/test.npz'

train = np.load(train_path)
test = np.load(test_path)

In [ ]:
# Get the train and test data
train_data = train['data']
train_labels = train['labels']

test_data = test['data']
test_labels = test['labels']

In [ ]:
# Data generator allows to transform each batch in training and evaluate the test set after each epoche
class DataGenerator(utilities.BaseDataGenerator):
    # There is no transformation needed, but it is still convenient to use a generator here
    # since using a tensorflow dataset would use more memory than is available on my machine
    def transform(self, X: np.ndarray) -> np.ndarray:
        return X

In [ ]:
# Initialize data generators
train_generator = DataGenerator(train_data, train_labels, batch_size=BATCH_SIZE)
test_generator = DataGenerator(test_data, test_labels, batch_size=BATCH_SIZE)

In [ ]:
# Create model
model = keras.Sequential([
    keras.layers.Input(shape=(20, 500, 6)),

    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D((1, 2)),

    keras.layers.Conv2D(64, 4, activation='relu'),
    keras.layers.MaxPooling2D((1, 3)),

    keras.layers.Conv2D(72, 5, activation='relu'),
    keras.layers.MaxPooling2D((1, 3)),

    keras.layers.Conv2D(80, 6, activation='relu'),
    keras.layers.MaxPooling2D((1, 3)),

    keras.layers.Flatten(),

    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.7),
    keras.layers.Dense(4, activation='softmax'),
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model on the training set and evaluate the test set on the end of each epoche
history = model.fit(train_generator, batch_size=BATCH_SIZE, epochs=EPOCHS)